In [1]:
import sys
import os
sys.path.append(".")
os.environ['JUPYTER_PATH'] = '.'

# Globus Automate: Flows and Actions

## Flow Definition

* Flows composed of asynchronous *Action* invocation and wait for completion operations
* Each Action invocation reads from and contributes back to the *Flow State*

```json
{
  "Comment": "Sync a folder tree to a remote host via a DMZ intermediate and generate an identifier for the final location",
  "StartAt": "TransferSourceToDMZ",
  "States": {
    "TransferSourceToDMZ": {
      "Comment": "Copy from source to DMZ endpoint",
      "Type": "Action",
      "Resource": "https://actions.automate.globus.org/Transfer",
      "InputPath": "$.TransferSourceToDMZSpec",
      "ResultPath": "$.TransferSourceToDMZSpecResult",
      "RunAs": "User",
      "Next": "TransferSourceToDMZWait",
      "Catch": [
        {
        "ErrorEquals": ["States.ALL"],
          "Next": "CleanDMZ"
        }
      ]
    },
    "TransferSourceToDMZWait": {
      "Comment": "Wait for Copy from source to DMZ endpoint to complete",
      "Type": "ActionWait",
      "ResultPath": "$.TransferSourceToDMZSpecResult",
      "Next": "TransferDMZToDest",
      "Catch": [
        {
        "ErrorEquals": ["States.ALL"],
          "Next": "CleanDMZ"
        }
      ]
    },
    "TransferDMZToDest": {
      "Comment": "Copy from DMZ intermediate to Destination endpoint",
      "Type": "Action",
      "Resource": "https://actions.automate.globus.org/Transfer",
      "InputPath": "$.TransferDMZToDestSpec",
      "ResultPath": "$.TransferDMZToDestSpecResult",
      "RunAs": "User",
      "Next": "TransferDMZToDestWait",
      "Catch": [
        {
        "ErrorEquals": ["States.ALL"],
          "Next": "CleanDMZ"
        }
      ]
    },
    "TransferDMZToDestWait": {
      "Comment": "Wait for Copy from DMZ to Destination endpoint to complete",
      "Type": "ActionWait",
      "ResultPath": "$.TransferDMZToDestSpecResult",
      "Next": "CreateIdentifier",
      "Catch": [
        {
        "ErrorEquals": ["States.ALL"],
          "Next": "CleanDMZ"
        }
      ]
    },
    "CreateIdentifier": {
      "Comment": "Create an Identifier for the new data location",
      "Type": "Action",
      "Resource": "https://actions.automate.globus.org/IdentifierCreate",
      "InputPath": "$.IdentifierCreateSpec",
      "ResultPath": "$.IdentifierCreateSpecResult",
      "RunAs": "User",
      "Next": "CleanDMZ",
      "Catch": [
        {
        "ErrorEquals": ["States.ALL"],
          "Next": "CleanDMZ"
        }
      ]
    },
    "CleanDMZ": {
      "Comment": "Remove Intermediate Data stored in DMZ",
      "Type": "Action",
      "Resource": "https://actions.automate.globus.org/TransferDelete",
      "InputPath": "$.CleanDMZSpec",
      "ResultPath": "$.CleanDMZSpecResult",
      "RunAs": "User",
      "Next": "CleanDMZWait"
    },
    "CleanDMZWait": {
      "Comment": "Wait for DMZ Cleanup to complete",
      "Type": "ActionWait",
      "ResultPath": "$.CleanDMZSpecResult",
      "End": true
    }
  }
}

```

<img src="flow_example.png">

In [ ]:
from globus.automate.client import FlowsClient

flow_client = FlowsClient(flows_access_token)
flow_id = flow_client.deploy(flow_definition)
flow_definition = flow_client.get_flow(flow_id)

## All Actions are invoked with the same pattern via REST

1. run
2. status
3. release
4. (cancel)

Each step goverend by Authentication

In [3]:
import time
from globus.automate.client import get_access_token_for_scope, create_action_client

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"
TRANSFER_SCOPE = "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/globus_transfer_action_all"
transfer_request = {
    "source_endpoint_id": "go#ep1",
    "destination_endpoint_id": "go#ep2",
    "transfer_items": [
        {"source_path": "/~/file1.txt", "destination_path": "/~/file1_new.txt"}
    ],
}
transfer_token = get_access_token_for_scope(TRANSFER_SCOPE, CLIENT_ID)
transfer_client = create_action_client(
    "http://actions.automate.globus.org/Transfer", transfer_token
)
# transfer_action_description = transfer_client.introspect()
# print(transfer_action_description)
transfer_action_status = transfer_client.run(transfer_request)
transfer_action_id = transfer_action_status["action_id"]
while transfer_action_status["status"] not in ("SUCCEEDED", "FAILED"):
    print(f'ActionId: {transfer_action_id} Status: {transfer_action_status["status"]}')
    time.sleep(2)
    transfer_action_status = transfer_client.status(transfer_action_id)

transfer_action_status = transfer_client.release(transfer_action_id)
print(f"Final Complete Status: {transfer_action_status.data}")


ActionId: 17oYjeP7IkoQ6 Status: ACTIVE
Final Complete Status: {'action_id': '17oYjeP7IkoQ6', 'creator_id': 'urn:globus:auth:identity:b44bddda-d274-11e5-978a-9f15789a8150', 'details': {'DATA_TYPE': 'task', 'bytes_checksummed': 0, 'bytes_transferred': 4, 'canceled_by_admin': None, 'canceled_by_admin_message': None, 'command': 'API 0.10', 'completion_time': '2018-11-21 22:04:50+00:00', 'deadline': '2018-11-22 22:04:49+00:00', 'delete_destination_extra': False, 'destination_endpoint': 'go#ep2', 'destination_endpoint_display_name': 'Globus Tutorial Endpoint 2', 'destination_endpoint_id': 'ddb59af0-6d04-11e5-ba46-22000b92c6ec', 'directories': 0, 'effective_bytes_per_second': 2, 'encrypt_data': False, 'event_link': {'DATA_TYPE': 'link', 'href': 'task/7625fa8a-edd9-11e8-8cb3-0a1d4c5c824a/event_list?format=json', 'rel': 'child', 'resource': 'event list', 'title': 'child event list'}, 'fatal_error': None, 'faults': 0, 'files': 1, 'files_skipped': 0, 'files_transferred': 1, 'history_deleted': Fal

## Flows invoked as Actions

In [ ]:
flow_request = {
    'source_endpoint': 'lab#endpoint',
    'dmz_endpoint': 'dmz#staging',
    'destination_endpoint': 'science#archive',
    'path': '/path/to/data',
    'identifier_metadata': {
        'creator': 'John Doe',
        'title': 'Science Dataset'
    }
}
flows_token = get_access_token_for_scope(FLOWS_SCOPE)
flows_client = ActionClient(f'https://flows.automate.globus.org/flows/{flow_id}', flows_token)
flow_action_status = flow_client.run(flow_request)
flow_action_status = flow_client.status(flow_action_status['action_id'])
final_status = flow_client.release(flow_action_status['action_id'])

## Action Implementation Helper in Automate SDK

In [ ]:
from globus.automate.common import AbstractActionProvider, ActionInstance, AuthState


class HelloWorldActionProvider(AbstractActionProvider):
    # Declare the format for inputs to this Action for use in Action Introspection
    # and for validation of input by the service
    request_body_schema = {
        "type": "object",
        "properties": {
            "echo_string": {"type": "string"},
            "sleep_time": {"type": "integer"},
        },
        "additionalProperties": False,
    }

    def __init__(self, *args, **kwargs):
        # Set the Actions REST API URL location on the server
        self.url_prefix = kwargs.get("url_prefix", "HelloWorld")
        
        # Define properties needed to do Authentication of requests via Globus Auth
        self.globus_auth_client_id = kwargs.get(
            "globus_auth_client_id", "5fac2e64-c734-4e6b-90ea-ff12ddbf9653"
        )
        self.globus_auth_client_name = kwargs.get(
            "globus_auth_client_name", "hello_world_action_provider"
        )
        self.globus_auth_client_secret = kwargs.get("globus_auth_client_secret")
        self.globus_auth_scope = kwargs.get(
            "globus_auth_scope",
            (
                "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/all"
            ),
        )
        
        # Set properties for Action Introspection
        self.title = "Hello World"
        self.subtitle = "An Action responding Hello to an input value"
        self.visible_to = kwargs.get("visible_to", ["public"])
        self.administered_by = kwargs.get("administered_by", ["foo@bar.com"])
        self.admin_contact = kwargs.get("admin_contact", "support@globus.org")
        self.synchronous = False
        self.log_supported = False
        self.runnable_by = kwargs.get("runnable_by", ["public"])
        self.input_schema = HelloWorldActionProvider.request_body_schema
        super(HelloWorldActionProvider, self).__init__(*args, **kwargs)

    def _action_done(self, action: ActionInstance) -> bool:
        """
        Helper for determining when a request with sleep_time in the request is
        completed.
        """
        if "sleep_time" not in action.request_body:
            return True
        else:
            now = datetime.datetime.now()
            run_length = (now - action.start_time).total_seconds()
            return run_length > int(action.request_body["sleep_time"])

    def run_action(self, action: ActionInstance, auth_state: AuthStatae) -> ActionInstance:
        # Callback for starting a new Action
        action.action_id = self.generate_actionid()
        action.details = {"Hello": "World"}
        if "echo_string" in action.request_body:
            action.details["hello"] = action.request_body["echo_string"]
        if self._action_done(action):
            action.status = "SUCCEEDED"
            action.completion_time = datetime.datetime.now()
        else:
            action.status = "ACTIVE"
        return action

    def check_status(self, action: ActionInstance, auth_state: AuthState) -> ActionInstance:
        # Callback for a user status check
        if self._action_done(action):
            action.status = "SUCCEEDED"
            action.completion_time = datetime.datetime.now()
        else:
            action.status = "ACTIVE"
        return action

    def cancel_action(self, action: ActionInstance, auth_state: AuthState) -> ActionInstance:
        # Callback for cancel
        action.status = 'FAILED'
        action.completion_time = datetime.datetime.now()
        return action

# Create the flask app, sqlalchemy engine and configure the action on the flask app
flask = Flask(__name__)
db_engine = sqlalchemy.create_engine(db_uri)
hello_world_provider = HelloWorldActionProvider()
hello_world_provider.set_flask_routes(flask, db_engine)